<a href="https://colab.research.google.com/github/kabanyasu/safie-colab/blob/master/%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#ファイル場所の指定
import os

path = "/content/drive/MyDrive/1011_セーフィー株式会社/pjt_人件費/人件費配賦資料"
os.chdir(path)
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1TQ5vITu2g3s-9F2I1PirGz-g7baa9H92/1011_セーフィー株式会社/pjt_人件費/人件費配賦資料


## 支給控除一覧表を部門配賦を行う形に変換する

In [5]:
import pandas as pd

#pathの指定
path = '個人コード順貼付用.xlsx'


#excelデータの読み込み
#支給控除一覧表を読み込む
payloll_list = pd.read_excel(path,header = None)  #1行目をcolumとして読み込まない

#1行目は必要ないので、削除
payloll_list = payloll_list.drop(1,axis=0)
#インデックスの再設定
payloll_list = payloll_list.reset_index(drop=True)

#nanをとりあえず空白に変換（下のfor文でエラーが出るから）
sp = " "  #のちに使うので変数で追えるようにする
payloll_list.iloc[1][0] =sp  #1行目の0番目の要素を半角空白記号へ


#「【」が一文字目についている番号を抽出
s = 0
drop_columns = []

#1行目の要素をどんどん読み込む
for i in payloll_list.iloc[1]:
  #「【」で始まるとき、その列は不必要なので、削除したい
  if i[0] == "【":
    #「【」で始まる列番号をメモる
    drop_columns.append(s)
  s = s+1

#メモった列番号をもとに、対応する列を削除
for i in drop_columns:
  payloll_list = payloll_list.drop([i],axis=1)

#転置
payloll_list = payloll_list.T

#氏名列のカラムのセット
payloll_list.iloc[0][1] = "氏名"
#1行目をカラムに入れる
payloll_list.columns = payloll_list.iloc[0]
#カラムに入れたのち、1行目を消す
payloll_list = payloll_list.drop([0],axis = 0)


#給料及び手当
#給与及び手当の加算項目
payloll_list['給料及び手当'] = payloll_list[['基本給', '固定残業手当', '調整給', '残業手当', 
                       '紹介手当'	, '特別手当', '賞金', '調整金（非課税）', '遡及支給', '精算金', '課税通勤手当','管理職手当','近距離手当']].sum(axis=1)
#給与及び手当の減算項目
payloll_list['給料及び手当'] = payloll_list['給料及び手当'] -payloll_list['減額金']

#雑給
payloll_list['雑給'] = payloll_list[['基本給（ﾊﾞｲﾄ）','ﾊﾞｲﾄｲﾝｾﾝﾃｨﾌﾞ']].sum(axis=1)

#福利厚生費
##福利厚生費がなかったため左辺省略
##「慶弔見舞金」→「慶弔見舞」に変更
payloll_list['福利厚生費'] = payloll_list[['慶弔見舞']].sum(axis=1)

#旅費交通費
payloll_list['旅費交通費'] = payloll_list['通勤手当']

#給料及び手当(残業代)
payloll_list["給料及び手当(残業代)"] = payloll_list[['残業手当']].sum(axis=1)

#社会保険料(健康保険料+介護保険料)
payloll_list["社会保険料(健康保険料+介護保険料)"] = payloll_list[["健康保険料","介護保険料"]].sum(axis=1)

#厚生年金保険料
payloll_list["厚生年金保険料"] = payloll_list[["厚生年金保険料"]].sum(axis=1)

#課税支給額
##そのままでOK

#氏名の行のカラム名を設定
payloll_list = payloll_list.rename(columns = {sp:"氏名"})

#旅費を含めた給与データ
payloll_list = payloll_list[["部門名称","氏名","役員報酬","給料及び手当","雑給","福利厚生費","旅費交通費","給料及び手当(残業代)","社会保険料(健康保険料+介護保険料)","厚生年金保険料","課税支給額"]]

#indexの整理（0から順に表示）
payloll_list = payloll_list.reset_index(drop=True)

payloll_list

,部門名称,氏名,役員報酬,給料及び手当,雑給,福利厚生費,旅費交通費,給料及び手当(残業代),社会保険料(健康保険料+介護保険料),厚生年金保険料,課税支給額
0,BU1営業部,本多 桃子,0,777023.0,0.0,0.0,0,20356.0,28900.0,59475.0,777023
1,BU1営業部,村下 善恵,0,696667.0,0.0,0.0,4896,0.0,29400.0,51240.0,696667
2,BU1営業部,齋藤 凛,0,466515.0,0.0,0.0,12420,0.0,19975.0,43005.0,470015
3,BU1営業部,中根 ひとみ,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
4,BU1営業部,永沢 篤,0,1000000.0,0.0,0.0,6408,0.0,43775.0,59475.0,1002500
...,...,...,...,...,...,...,...,...,...,...,...
308,社長室,中島 早香,550000,0.0,0.0,0.0,13632,0.0,26250.0,45750.0,550000
309,社長室,髙橋 由佳,0,375000.0,0.0,0.0,11424,0.0,16150.0,34770.0,375250
310,社長室,工藤 克己,400000,0.0,0.0,0.0,0,0.0,0.0,0.0,400000
311,社長室,岡田 淳,420000,0.0,0.0,0.0,0,0.0,0.0,0.0,420000


## 開発工数シートをテーブル状に変換する
- 開発工数シートの読み込み
- メールアドレスと従業員一覧の対応を行う
- メールアドレスをキーに、従業員一覧と開発工数シートを結合する

In [6]:
#開発工数シートを読み込む
path = "開発工数シート.xlsx"
#1行目をcolumnsとして読み込まない
before_man_hour_list = pd.read_excel(path,header = None)
#NaNを0に変換
before_man_hour_list = before_man_hour_list.fillna(0)


#不必要な部分の削除
before_man_hour_list = before_man_hour_list.drop([0,1,2], axis=0)  #上から3行消す
before_man_hour_list = before_man_hour_list.drop(before_man_hour_list.tail(2).index,axis=0)  #下から2行消す

#列番号の整理
before_man_hour_list = before_man_hour_list.reset_index(drop=True)

#1行目をカラムに入れる
before_man_hour_list.columns = before_man_hour_list.iloc[0]

#カラムに入れたのち、不必要となる1行目を消す
before_man_hour_list = before_man_hour_list.drop([0],axis = 0)

#カラムの「行ラベル」を「メールアドレス」に変更する
before_man_hour_list = before_man_hour_list.rename(columns = {"行ラベル":"メールアドレス"})

before_man_hour_list

,メールアドレス,0:保守、運用,47:通知プラットフォーム,49:次世代カメラ開発(SafieOne）,52:解析PF,53:AIApp,61:Enterprise Toolマイグレーション,63:在庫システム,64:Entrance2,65:鹿島建設DX PJ,...,78:(旧62ソフトフェアPJ 代理店対応)⇒ソフトフェアPJ Agency Tool,79:(旧66)ソフトフェアPJ データ分析基盤,80:Pocket3,81:多台数対応,83:Pocket2+,84:メッシュWiFi,85:シェアプラス,86:Local Viewer,(空白),総計
1,a-baba@safie.jp,6282,0,0,0,0,0,0,856,0,...,0,0,0,3023,0,0,0,0,0,10161
2,a-hirakawa@safie.jp,0,0,0,0,0,0,0,0,0,...,0,10497,0,0,0,0,0,0,0,10497
3,a-ninomiya@safie.jp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9617
4,a-ohkawa@safie.jp,0,0,1113,0,5630,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11049
5,a-shimoyamada@safie.jp,10414,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,y-sasaki@safie.jp,0,0,0,0,0,0,0,0,0,...,0,0,0,7556,0,0,0,0,0,10592
86,y-sato@safie.jp,9873,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9873
87,y-shimada@safie.jp,1891,0,0,0,0,0,0,0,0,...,0,0,1840,0,3547,0,0,0,0,9195
88,y-uematsu@safie.jp,10225,0,1156,0,1292,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12673


##メールアドレスと氏名の対応表作成
- 従業員一覧貼付.xlsxを読み込む
- 従業員一覧貼付.xlsxからメールアドレスと氏名を抽出する
- メールアドレスと氏名を新たなデータフレームに格納する

In [7]:
#支給控除一覧表と開発工数シートをつなぐ「メールアドレスと氏名」の対応表を作成

#従業員一覧を読み込む
workr_list = pd.read_excel('従業員一覧貼付.xlsx')

#新たなデータフレームを作成
name_address_dict = pd.DataFrame()

#従業員一覧の必要な部分のみを抽出
name_address_dict[0] = workr_list["メールアドレス"]
name_address_dict[1] = workr_list["氏名"]

#カラム名を設定
name_address_dict.columns = ["メールアドレス","氏名"]


name_address_dict

,メールアドレス,氏名
0,sadox@safie.jp,佐渡島 隆平
1,zksm@safie.jp,下崎 守朗
2,mkazuma@safie.jp,森本 数馬
3,manaty@safie.jp,藤木 麻那
4,yok@safie.jp,横堀 雅之
...,...,...
344,takaura.hiromi@safie.jp,高浦 浩実
345,qc-staff01@safie.jp,飯塚 新
346,mizumachi.kanako@safie.jp,水町 佳奈子
347,nagao.tomoko@safie.jp,長尾 智子


## 開発工数シートのメールアドレスを元に、氏名を対応させる
- 開発工数シートの主キーは、メールアドレスになっている
- 支給控除一覧表と結合したい
- メールアドレスをキーに、氏名を対応させる

In [8]:
#メールアドレス、名前の対応表から名前を探す

#name_address_dictに、before_man_hour_listの「"メールアドレス"」一致部分を結合
man_hour_list = pd.merge(name_address_dict, before_man_hour_list,on="メールアドレス",how="left")

man_hour_list

,メールアドレス,氏名,0:保守、運用,47:通知プラットフォーム,49:次世代カメラ開発(SafieOne）,52:解析PF,53:AIApp,61:Enterprise Toolマイグレーション,63:在庫システム,64:Entrance2,...,78:(旧62ソフトフェアPJ 代理店対応)⇒ソフトフェアPJ Agency Tool,79:(旧66)ソフトフェアPJ データ分析基盤,80:Pocket3,81:多台数対応,83:Pocket2+,84:メッシュWiFi,85:シェアプラス,86:Local Viewer,(空白),総計
0,sadox@safie.jp,佐渡島 隆平,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,zksm@safie.jp,下崎 守朗,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mkazuma@safie.jp,森本 数馬,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,manaty@safie.jp,藤木 麻那,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,yok@safie.jp,横堀 雅之,6511,0,0,0,0,0,0,2227,...,0,0,0,0,0,0,0,0,0,10724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,takaura.hiromi@safie.jp,高浦 浩実,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,qc-staff01@safie.jp,飯塚 新,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,mizumachi.kanako@safie.jp,水町 佳奈子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,nagao.tomoko@safie.jp,長尾 智子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 給与リストと工数開発シートの結合
- 2つのシートの氏名に含まれる空白を削除する
- 氏名をもとに、給与リストと開発工数シートを結合する

In [9]:
# 結合のため両者のデータの空白をなくす
payloll_list['氏名'] = payloll_list['氏名'].str.replace(' ', '')
payloll_list['氏名'] = payloll_list['氏名'].str.replace('　','')

man_hour_list['氏名'] = man_hour_list['氏名'].str.replace(' ', '')
man_hour_list['氏名'] = man_hour_list['氏名'].str.replace('　','')

#支給控除一覧表(payloll_list)と、工数開発シート(man_hour_list)のmerge
#「氏名」をkeyにして、payloll_listにman_hour_listの要素を結合する
merge_payloll_man_hour = pd.merge(payloll_list,man_hour_list,on = "氏名",how ="left")

#NaNを0にする
merge_payloll_man_hour = merge_payloll_man_hour.fillna(0)

merge_payloll_man_hour

,部門名称,氏名,役員報酬,給料及び手当,雑給,福利厚生費,旅費交通費,給料及び手当(残業代),社会保険料(健康保険料+介護保険料),厚生年金保険料,...,78:(旧62ソフトフェアPJ 代理店対応)⇒ソフトフェアPJ Agency Tool,79:(旧66)ソフトフェアPJ データ分析基盤,80:Pocket3,81:多台数対応,83:Pocket2+,84:メッシュWiFi,85:シェアプラス,86:Local Viewer,(空白),総計
0,BU1営業部,本多桃子,0,777023.0,0.0,0.0,0,20356.0,28900.0,59475.0,...,0,0,0,0,0,0,0,0,0,0
1,BU1営業部,村下善恵,0,696667.0,0.0,0.0,4896,0.0,29400.0,51240.0,...,0,0,0,0,0,0,0,0,0,0
2,BU1営業部,齋藤凛,0,466515.0,0.0,0.0,12420,0.0,19975.0,43005.0,...,0,0,0,0,0,0,0,0,0,0
3,BU1営業部,中根ひとみ,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,BU1営業部,永沢篤,0,1000000.0,0.0,0.0,6408,0.0,43775.0,59475.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,社長室,中島早香,550000,0.0,0.0,0.0,13632,0.0,26250.0,45750.0,...,0,0,0,0,0,0,0,0,0,0
309,社長室,髙橋由佳,0,375000.0,0.0,0.0,11424,0.0,16150.0,34770.0,...,0,0,0,0,0,0,0,0,0,0
310,社長室,工藤克己,400000,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
311,社長室,岡田淳,420000,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#中身の確認
#メールアドレスが取れている人数をみる
merge_payloll_man_hour[merge_payloll_man_hour['メールアドレス']!=0].count()  #276人がメールアドレス有,開発工数のデータは82人が該当

#問題点
#理論上はok。完成したcsvファイルを見ると開発工数データの取得を、できていない人がいる
#開発工数シート取得時点での行数は89行（before_man_hour_list）、加工後の行数は82行(merge_payloll_man_hour)

#氏名で結合する際に、「何かが起こった」と考えるのが妥当

部門名称                                           276
氏名                                             276
役員報酬                                           276
給料及び手当                                         276
雑給                                             276
福利厚生費                                          276
旅費交通費                                          276
給料及び手当(残業代)                                    276
社会保険料(健康保険料+介護保険料)                             276
厚生年金保険料                                        276
課税支給額                                          276
メールアドレス                                        276
0:保守、運用                                        276
47:通知プラットフォーム                                  276
49:次世代カメラ開発(SafieOne）                          276
52:解析PF                                        276
53:AIApp                                       276
61:Enterprise Toolマイグレーション                     276
63:在庫システム                                      276
64:Entrance2                   

##部門配賦の計算に必要なカラムを準備する

In [11]:
#部門配賦した給与を計算するために、以下のカラムを作成

#メールアドレスと氏名の列を取り除く
calc_columns = man_hour_list.drop(["メールアドレス","氏名"],axis=1).columns
calc_columns

Index(['0:保守、運用', '47:通知プラットフォーム', '49:次世代カメラ開発(SafieOne）', '52:解析PF',
       '53:AIApp', '61:Enterprise Toolマイグレーション', '63:在庫システム', '64:Entrance2',
       '65:鹿島建設DX PJ', '68:i-PRO製PTZ', '69:MyPageと契約管理システム',
       '71:Pocket2 位置情報連携など機能追加/改良', '72:Safie API 組込販売改善',
       '73:SafieManagerV3', '74:MyPortal', '75:(旧58)ソフトフェアPJ Open API (2022)',
       '76:(旧59)ソフトフェアPJ Mobile App (2022)', '77:(旧60)ソフトフェアPJ Viewer (2022)',
       '78:(旧62ソフトフェアPJ 代理店対応)⇒ソフトフェアPJ Agency Tool',
       '79:(旧66)ソフトフェアPJ データ分析基盤', '80:Pocket3', '81:多台数対応', '83:Pocket2+',
       '84:メッシュWiFi', '85:シェアプラス', '86:Local Viewer', '(空白)', '総計'],
      dtype='object')

##部門配賦の計算をする
- 計算例 →  「iさんの給与及び手当 / iさんの総計 * iさんの保守・運用」
- 計算を全員に対して行い、output.csvとして保存する

In [12]:
#iさんの給与及び手当 / その人の総計 * iさん0:保守、運用というように計算していく
for i in ['給料及び手当', '雑給', '福利厚生費', '旅費交通費' ,'給料及び手当(残業代)', '社会保険料(健康保険料+介護保険料)','厚生年金保険料', '課税支給額']:
  for j in calc_columns:
    merge_payloll_man_hour[i+ '_' + j] = merge_payloll_man_hour[i] / merge_payloll_man_hour['総計'] * merge_payloll_man_hour[j]

merge_payloll_man_hour.count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


部門名称                     313
氏名                       313
役員報酬                     313
給料及び手当                   313
雑給                       313
                        ... 
課税支給額_84:メッシュWiFi         82
課税支給額_85:シェアプラス           82
課税支給額_86:Local Viewer     82
課税支給額_(空白)                82
課税支給額_総計                  82
Length: 264, dtype: int64

In [13]:
#csv形式で保存
merge_payloll_man_hour.to_csv('output_変更後.csv')

## 計算対象シートを用いて、部門ごとの値を計算する
完成形は
- 列→「給料及び手当」、「雑給」
- 行→「47:通知プラットホーム」、「49:次世代カメラ開発(SafieOne）」

となる。

####ピボットテーブルの機能を使い、以下のようにする
- indexを各事業部
- columnsを工数から算出された数値を反映

〈テーブルの中身〉
- index→「〇〇部」or「○○室」
- columns→「給料及び手当_47:通知プラットフォーム」、「雑給_49:次世代カメラ開発(SafieOne）」など

####不必要な部分を削り、それを転置する

〈テーブルの中身（転置後）〉
- columns→「〇〇部」or「○○室」
- index→「給料及び手当_47:通知プラットフォーム」、「雑給_49:次世代カメラ開発(SafieOne）」など



In [14]:
import pandas as pd

category_calc_sheet = pd.read_csv("output_変更後.csv")

#部門ごとのピボットテーブルの作成
#デフォルトは平均になっている（aggfunc="mean"）のため、aggfunc="sum"に変える
category_calc_sheet = category_calc_sheet.pivot_table(index="部門名称", values= category_calc_sheet.columns,aggfunc="sum")

#ピボットテーブル内の、不必要な部分の削除
category_calc_sheet = category_calc_sheet.loc["BU1プロダクト部":"経理部","厚生年金保険料_(空白)":"雑給_総計"]
category_calc_sheet = category_calc_sheet.T

category_calc_sheet

部門名称,BU1プロダクト部,BU1営業部,BU2プロダクト部,BU2営業部,BU3プロダクト部,BU3営業部,アライアンス戦略室,エンジニアリングオフィス,カスタマーコミュニケーション部,デザインセンター,...,プロダクト基盤開発部,マーケティング部,人事総務部,内部監査室,品質管理部,業務システム部,法務部,社長室,経営企画部,経理部
厚生年金保険料_(空白),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
厚生年金保険料_0:保守、運用,206070.085751,0.0,170915.817216,0.0,73142.086217,0.0,0.0,29292.436622,0.0,291443.739908,...,484295.631013,0.0,0.0,0.0,0.0,141628.953096,0.0,0.0,0.0,0.0
厚生年金保険料_47:通知プラットフォーム,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,7099.716074,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
厚生年金保険料_49:次世代カメラ開発(SafieOne）,163873.055833,0.0,0.000000,0.0,0.000000,0.0,0.0,11839.594151,0.0,19653.715205,...,15374.051174,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
厚生年金保険料_52:解析PF,5330.197183,0.0,18274.618585,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,243164.860553,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
雑給_83:Pocket2+,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
雑給_84:メッシュWiFi,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
雑給_85:シェアプラス,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
雑給_86:Local Viewer,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


##横計の計算をする
- 上で出力されているテーブルを利用し、プロジェクトごとの科目の金額を横計で集計する。
- 不必要な部分をカット（プロジェクト名が記入されているキー部分と横計金額以外を削除）。
- 科目ごとに分けて、別々のデータフレームとした上で、1つのリストに格納する。
- リストの中に科目ごとに集計したデータフレームが7個入っている。

科目ごとに分かれているため、科目ごとのプロジェクトの金額を見たいときは
- category_calc_sheet_list[0]["給料及び手当_"]
- category_calc_sheet_list[1]["雑給"]

これで確認できる。

In [15]:
#リストの中に複数のデータフレームを格納
category_calc_sheet_list = []

#部門名称と、各金額が表示されるデータフレーム（2列のデータフレーム）を作成して、それをリスト内に入れる
k = ["給料及び手当_","雑給_","福利厚生費_","旅費交通費_","厚生年金保険料_"]

for i in k:
    #kの文字が含まれているindexを探して、その行だけを抽出
    add_category_sheet = category_calc_sheet.query("index.str.contains('{}')".format(i), engine='python')
    #それらを横で合計する
    #「給与及び手当の、保守・運用合計のセル」、「給与及び手当の、通知プラットフォーム合計のセル」というように、新たな列を作成する
    add_category_sheet[i] = add_category_sheet.sum(axis = 1)
    add_category_sheet = add_category_sheet.reset_index()
    add_category_sheet = add_category_sheet[["index",i]]
    add_category_sheet.columns = ["部門名称",i]
    #リストに都度appendで格納する。1つのリストの中に5つのデータフレームが格納される
    category_calc_sheet_list.append(add_category_sheet)


#鍵括弧や加算記号"+"などとの相性が悪いため、特徴箇所で抽出して、少しだけ違う方法で抽出
#内容は上のfor文と同じ
n = ["残業代","健康保険料"]

for i in n:
    #kの文字が含まれているindexを探して、その行だけを抽出
    add_category_sheet = category_calc_sheet.query("index.str.contains('{}')".format(i), engine='python')
    add_category_sheet = add_category_sheet.reset_index()
    add_category_sheet = add_category_sheet.drop(0,axis=0)
    #それらを横で合計する
    add_category_sheet[i] = add_category_sheet.sum(axis = 1)
    add_category_sheet = add_category_sheet.reset_index()
    add_category_sheet = add_category_sheet[["index",i]]
    add_category_sheet.columns = ["部門名称",i]
    #リストに追加で2つのデータフレームを格納。7個のデータフレームが一つのリストに格納されている状態。
    category_calc_sheet_list.append(add_category_sheet)

category_calc_sheet_list[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


,部門名称,給料及び手当_
0,給料及び手当_(空白),0.000000e+00
1,給料及び手当_0:保守、運用,1.851120e+07
2,給料及び手当_47:通知プラットフォーム,8.817186e+04
3,給料及び手当_49:次世代カメラ開発(SafieOne）,2.634691e+06
4,給料及び手当_52:解析PF,3.151816e+06
5,給料及び手当_53:AIApp,3.704739e+06
6,給料及び手当_61:Enterprise Toolマイグレーション,1.368175e+05
7,給料及び手当_63:在庫システム,5.625000e+05
8,給料及び手当_64:Entrance2,2.817039e+06
9,給料及び手当_65:鹿島建設DX PJ,6.712390e+05


##1つのデータフレームにまとめる・最終段階
- 1つのリストの中にある、7つのデータフレームを呼び出して、1つのテーブルにまとめる。
- 各科目の数値部分だけを抜き取って、1枚目のテーブルに結合するようなイメージ
- columnsは各科目名
-indexは各プロジェクト名が入る

In [40]:
m = 0

#移行先のデータ用意
category_calc_sheet_comp = pd.DataFrame()
category_calc_sheet_comp["部門名称"] = category_calc_sheet_list[0]["部門名称"]

#二つのリストを足して、一つのリストにまとめる
#k = ["給料及び手当_","雑給_","福利厚生費_","旅費交通費_","厚生年金保険料_"]
#n = ["残業代","健康保険料"]
category_calc_sheet_comp_col = k+n
#リストにある7つのデータフレームを1つのデータフレームにまとめて、1つのシート上で完結させる
for j in category_calc_sheet_comp_col:
    category_calc_sheet_comp[j] = category_calc_sheet_list[m][j]
    m = m +1


category_calc_sheet_comp = category_calc_sheet_comp.drop([0,27] ,axis=0)
category_calc_sheet_comp = category_calc_sheet_comp.reset_index(drop=True)

#部門名称列の整理
#「給料及び手当_0:保守、運用」→「0:保守、運用」
category_calc_sheet_comp_col = category_calc_sheet_comp["部門名称"].str.strip("給料及び手当_.")
category_calc_sheet_comp["部門名称"] = category_calc_sheet_comp_col

#category_calc_sheet_comp.to_csv('部門名称データ.csv')
category_calc_sheet_comp

,部門名称,給料及び手当_,雑給_,福利厚生費_,旅費交通費_,厚生年金保険料_,残業代,健康保険料
0,0:保守、運用,1.851120e+07,0.0,-50000.0,124370.832049,1.439794e+06,60923.211670,777896.236019
1,47:通知プラットフォーム,8.817186e+04,0.0,0.0,0.000000,7.099716e+03,29.403483,3297.682329
2,49:次世代カメラ開発(SafieOne）,2.634691e+06,0.0,0.0,29239.620643,2.107404e+05,60051.792897,120023.413808
3,52:解析PF,3.151816e+06,0.0,0.0,15409.090610,2.667697e+05,10555.813608,134288.256817
4,53:AIApp,3.704739e+06,0.0,0.0,29062.011016,3.222381e+05,630.967481,162198.863016
5,61:Enterprise Toolマイグレーション,1.368175e+05,0.0,0.0,292.240037,1.053176e+04,142.263363,6568.890590
6,63:在庫システム,5.625000e+05,0.0,0.0,320.000000,5.124000e+04,0.000000,23800.000000
7,64:Entrance2,2.817039e+06,0.0,0.0,22894.111571,2.491482e+05,3180.289135,119702.097659
8,65:鹿島建設DX PJ,6.712390e+05,0.0,0.0,3245.421788,5.441369e+04,0.000000,31927.834226
9,68:i-PRO製PTZ,8.318552e+05,0.0,0.0,11590.010962,7.514254e+04,7.673575,43001.408769


##保守・運用を取得して結合
8つの部門の保守・運用をmaintenance_Operationとして、category_calc_sheet_compと結合させる

In [51]:
maintenance_Operation = category_calc_sheet[['BU1プロダクト部',
                                             'BU2プロダクト部',
                                             'BU3プロダクト部',
                                             'プラットフォームビジネス開発室',
                                             'プロダクト基盤開発部',
                                             'エンジニアリングオフィス',
                                             'デザインセンター',
                                             '業務システム部']]

maintenance_Operation = maintenance_Operation[maintenance_Operation.index.str.endswith('_0:保守、運用')]

maintenance_Operation = maintenance_Operation.T

maintenance_Operation.columns = ['厚生年金保険料',
                                 '旅費交通費',
                                 '社会保険料(健康保険料+介護保険料)',
                                 '福利厚生費',
                                 '給料及び手当(残業代)',
                                 '給料及び手当',
                                 '課税支給額','雑給']


project_name_list = []
#保守運用であることを付け足す
for project_name in maintenance_Operation.index:
    i = project_name + '_0:保守、運用'
    project_name_list.append(i)

maintenance_Operation.index = project_name_list

#※改善？
maintenance_Operation = maintenance_Operation[['給料及び手当',
                                               '雑給',
                                               '福利厚生費',
                                               '旅費交通費',
                                               '厚生年金保険料',
                                               '給料及び手当(残業代)',
                                               '社会保険料(健康保険料+介護保険料)']]


maintenance_Operation = maintenance_Operation.reset_index()
maintenance_Operation

maintenance_Operation

,index,給料及び手当,雑給,福利厚生費,旅費交通費,厚生年金保険料,給料及び手当(残業代),社会保険料(健康保険料+介護保険料)
0,BU1プロダクト部_0:保守、運用,2.739839e+06,0.0,0.0,27737.944372,206070.085751,22396.430737,125562.135744
1,BU2プロダクト部_0:保守、運用,2.142208e+06,0.0,0.0,11663.697281,170915.817216,555.295925,90638.376205
2,BU3プロダクト部_0:保守、運用,8.996212e+05,0.0,0.0,11971.163607,73142.086217,897.739348,35280.525942
3,プラットフォームビジネス開発室_0:保守、運用,4.784220e+05,0.0,0.0,0.000000,43005.000000,0.000000,19975.000000
4,プロダクト基盤開発部_0:保守、運用,6.144283e+06,0.0,-50000.0,50179.884705,484295.631013,34580.504499,245873.344946
5,エンジニアリングオフィス_0:保守、運用,4.234103e+05,0.0,0.0,1052.015883,29292.436622,666.375229,21461.419517
6,デザインセンター_0:保守、運用,4.039655e+06,0.0,0.0,15364.361598,291443.739908,530.011377,166858.196414
7,業務システム部_0:保守、運用,1.643761e+06,0.0,0.0,6401.764603,141628.953096,1296.854554,72247.237251


##給与伝票の形に変える
検証段階

In [ ]:
#部門ごとの最終的な形の作成
#「47:通知プラットフォーム」から「86:Local Viewer	」までの各費用を最終形態へ
output_s = category_calc_sheet_comp
output_s = output_s.drop(0,axis=0)
output_s = output_s.reset_index(drop=True)

output_s_bumon = pd.DataFrame()

#名称だけ切り離して数値部分を操作して、後で結合する
output_s_bumon["部門名称"] = output_s["部門名称"]
output_s = output_s.drop("部門名称",axis=1)

output_s = output_s.T

output_s = output_s.reset_index()

#番号関連は変数に+1ずつしていく方向で考える

a = output_s[["index",0]]
a
#各通知プラットフォームの部分を「for i in 部門名称が入っているやつ」で対応させる？
a["部門名称"]= "47:通知プラットフォーム"
a = a[["部門名称","index",0]]



b = output_s[["index",1]]

#各通知プラットフォームの部分を「for i in 部門名称が入っているやつ」で対応させる？
b["部門名称"]= "49:次世代カメラ開発(SafieOne）"
b = b[["部門名称","index",1]]

#カラムの名前を合わせる。ここは固定。
b.columns =["部門名称","index",0]

a = pd.concat([a,b])

a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,部門名称,index,0
0,47:通知プラットフォーム,給料及び手当_,8.817186e+04
1,47:通知プラットフォーム,雑給_,0.000000e+00
2,47:通知プラットフォーム,福利厚生費_,0.000000e+00
3,47:通知プラットフォーム,旅費交通費_,0.000000e+00
4,47:通知プラットフォーム,厚生年金保険料_,7.099716e+03
5,47:通知プラットフォーム,残業代,2.940348e+01
6,47:通知プラットフォーム,健康保険料,3.297682e+03
0,49:次世代カメラ開発(SafieOne）,給料及び手当_,2.634691e+06
1,49:次世代カメラ開発(SafieOne）,雑給_,0.000000e+00
2,49:次世代カメラ開発(SafieOne）,福利厚生費_,0.000000e+00


In [ ]:
# githubtest